# Harmony autoscaling metrics

This notebook shows performance metrics for three different workload runs. Each run was executed using our workload driver with 50 concurrent requests at all times. Each run lasted one hour.

1.  The first run was prior to implementation of any autoscaling. The harmony environment was a fixed environment with 2 pods for each service and 2 worker nodes.
2.  The second run was with autoscaling and fair queueing enabled. Each service could scale up from 1 to 10 pods and would scale up a new pod for every 10 work items currently queued. Worker nodes scale up based on CPU load and could scale up from 2 nodes up to 6 nodes.
3.  The third run was also with autoscaling and fair queueing enabled. Each service could scale up from 1 to 50 pods and would scale up a new pod for every work item queued. Same as with the previous run worker nodes scaled up based on CPU load and could scale up from 2 nodes up to 6 nodes.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from harmony.charts import create_bar, create_data_frame, display_bar_chart

Load the workload run reports for the three runs

In [ ]:
data_files_dict = {
    'Pre-autoscaling': 'examples/pre-autoscaling.csv',
    'Autoscale 10 pods': 'examples/10-pods-per-service-1-pod-per-10-queued.csv',
    'Autoscale 50 pods': 'examples/50-pod-per-service-1-pod-per-queued.csv'
}

## Average response time for single granule synchronous requests

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])
    df['Average Seconds'] = df['Average Response Time'] / 1000
    df = df.loc[df['Type'] == 'GET']
    df = df.loc[df['Name'] != 'async request started']
    bar_object = create_bar(label, df, 'Average Seconds')
    bar_objects.append(bar_object)

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Average response time for synchronous single granule requests', yaxis_title='Seconds')

## Number of failures for single granule synchronous requests
Note that almost all of the failures were 3 minute timeouts waiting for a response

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])
    df = df.loc[df['Type'] == 'GET']
    df = df.loc[df['Name'] != 'async request started']
    df['Failure Percentage'] = df['Failure Count'] / df['Request Count'] * 100
    bar_object = create_bar(label, df, 'Failure Percentage')
    bar_objects.append(bar_object)

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Failure percentage')

Prior to autoscaling and fair queueing synchronous requests for services that were also working on larger asynchronous requests would often time out.

## Average total request time for asynchronous requests

There are a few different types of asynchronous requests.

1. Requests that allow granules to be worked in parallel.
2. Requests that require sending all of the granules in a single request to a backend service (aggregation services).
3. Chained requests that allow multiple granules to be processed in parallel for each step, but each step needs to be executed serially.

In [ ]:
async_parallel_granule_requests = [
    '014: 50 granules Harmony Service Example: Bbox, Variable, and reformat',
    '009: 50 granules PODAAC L2SS Async Spatial and Temporal',
]

chained_requests = [
    '013: Chain L2SS to zarr'
]

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])
    df['Average Seconds'] = df['Average Response Time'] / 1000
    df = df.loc[df['Type'] == 'async_job']
    df = df.loc[df['Name'].isin(async_parallel_granule_requests)]
    bar_object = create_bar(label, df, 'Average Seconds')
    bar_objects.append(bar_object)

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Average response time for asynchronous requests with granules worked in parallel', yaxis_title='Seconds')

Requests that can be parallelized are completing faster. This chart is a good indicator that scaling is working well and has not hit some other resource bottleneck.

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])
    df['Average Seconds'] = df['Average Response Time'] / 1000
    df = df.loc[df['Type'] == 'async_job']
    df = df.loc[~df['Name'].isin(async_parallel_granule_requests + chained_requests + ['async request started'])]
    bar_object = create_bar(label, df, 'Average Seconds')
    bar_objects.append(bar_object)

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Average response time for asynchronous requests that cannot be worked in parallel', yaxis_title='Seconds')

Notice that the fair queueing policy prioritizing synchronous requests caused the minimal autoscaling with 10 pods per service to be worse for this class of requests when compared with the no autoscaling and no queue prioritaztion run. The focus on synchronous requests was delaying work on the asynchronous ones, so clearly we needed more resources when there are 50 concurrent requests in the system.

The 3rd and 4th requests are interesting - both for PODAAC concise. A two granule request completed quite a bit faster with the scaled up system, but the 50 granule request was slightly slower. My best interpretation is that there was less queueing in the scaled up system, but more resource contention - likely CPU, which caused the request to take longer. We should repeat the test to make sure it wasn't some difference on the download site, but if so it might mean we should increase the number of nodes or decrease the number of pods we allow. 

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])
    df['Average Seconds'] = df['Average Response Time'] / 1000
    df = df.loc[df['Type'] == 'async_job']
    df = df.loc[df['Name'].isin(chained_requests)]
    bar_object = create_bar(label, df, 'Average Seconds')
    bar_objects.append(bar_object)

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Average response time for chained asynchronous requests', yaxis_title='Seconds')

For the chained request there are three steps where work will be queued - query CMR, L2 subsetter, and zarr formatter. Since this request is for just a single granule it shows that there is less time spent in queues on the scaled up system.

## System throughput

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])
    df = df.loc[df['Name'] != 'async request started']
    bar_object = create_bar(label, df, 'Request Count')
    bar_objects.append(bar_object)

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Number of requests', yaxis_title='')

Overall system throughput is significantly better in the scaled up system. We may be able to scale pods and worker nodes more before finding the next resource bottleneck.

In [ ]:
bar_objects = []
for label in data_files_dict:
    df = create_data_frame(data_files_dict[label])

    df['Granule Count'] = [1, 1, 48, 1, 1, 50, 1, 2, 1, 1, 50, 50, 0]
    throughput = sum(df['Granule Count'] * df['Request Count'])
    
    data = [[label, throughput]]
 
    df = pd.DataFrame(data, columns = ['Name', 'Num Granules'])
    bar_object = create_bar(label, df, 'Num Granules')
    bar_objects.append(bar_object)
    

fig = go.Figure(data=bar_objects)
display_bar_chart(fig, 'Granule throughput', yaxis_title='Granules')